In [1]:
from sklearn.datasets import fetch_lfw_pairs

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import Perceptron

In [2]:
lfw_dataset_train = fetch_lfw_pairs(subset='train', resize = 1)
lfw_dataset_test = fetch_lfw_pairs(subset='test', resize = 1)

In [3]:
X_train = lfw_dataset_train.pairs
y_train = lfw_dataset_train.target

X_test = lfw_dataset_test.pairs
y_test = lfw_dataset_test.target

In [4]:
# x_train_1 = np.load("../npembedding/train_data_1.npy")
# x_train_2 = np.load("../npembedding/train_data_2.npy")

# x_test_1 = np.load("../npembedding/test_data_1.npy")
# x_test_2 = np.load("../npembedding/test_data_2.npy")

In [5]:
# y_train = np.load("../npembedding/y_train_data.npy")
# y_test = np.load("../npembedding/y_test_data.npy")

In [6]:
x_train_1 = X_train[:, 0]  # x_train_1.shape is (60000, 28, 28)
x_train_2 = X_train[:, 1]

x_test_1 = X_test[:, 0]  # x_val_1.shape = (60000, 28, 28)
x_test_2 = X_test[:, 1]

In [7]:
x_train_1.shape

(2200, 125, 94)

In [9]:
x_train_1[0].shape

(125, 94)

In [10]:
x_train_1[0]

array([[0.32418302, 0.3124183 , 0.28366014, ..., 0.48235297, 0.49411765,
        0.48496735],
       [0.3124183 , 0.29281047, 0.26797387, ..., 0.47320262, 0.4771242 ,
        0.46535948],
       [0.30457518, 0.2771242 , 0.27189544, ..., 0.45751634, 0.44705883,
        0.4392157 ],
       ...,
       [0.6666667 , 0.65882355, 0.6339869 , ..., 0.6418301 , 0.6666667 ,
        0.73594767],
       [0.6392157 , 0.65882355, 0.6562092 , ..., 0.6431373 , 0.62614375,
        0.6313726 ],
       [0.5921569 , 0.6509804 , 0.6901961 , ..., 0.63398695, 0.59738564,
        0.5254902 ]], dtype=float32)

In [22]:
84/(255-0)

0.32941176470588235

In [21]:
84/(255-0)

0.32941176470588235

In [23]:
def flatten_arr(data):
    arr = []
    for image in data:
        arr.append(image.flatten())
    return np.array(arr)

In [12]:
x_train_1 = flatten_arr(x_train_1)
x_train_2 = flatten_arr(x_train_2)

x_test_1 = flatten_arr(x_test_1)
x_test_2 = flatten_arr(x_test_2)

In [13]:
x_train_1.shape, x_train_2.shape, x_test_1.shape, x_test_2.shape

((2200, 11750), (2200, 11750), (1000, 11750), (1000, 11750))

In [29]:
# x_train_1 = x_train_1.flatten()
# x_train_2 = x_train_2.flatten()

# x_test_1 = x_test_1.flatten()
# x_test_2 = x_test_2.flatten()

In [30]:
# scaler = StandardScaler()
# def normalize(X_train,X_test):
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)
#     return X_train, X_test

  
# x_train_1, x_test_1 = normalize(x_train_1, x_test_1)
# x_train_2, x_test_2 = normalize(x_train_2, x_test_2)

In [24]:
# def difference(data1, data2):
#     sub = data1 - data2
#     return sub

# x_train_sub = difference(x_train_1, x_train_2)
# x_test_sub = difference(x_test_1, x_test_2)

In [25]:
# x_train_sub.shape, x_test_sub.shape

((2200, 11750), (1000, 11750))

In [26]:
# x_train_1 = x_train_1[:, 0]  # x_train_1.shape is (60000, 28, 28)
# x_train_2 = x_train_2[:, 1]

# x_test_1 = x_test_1[:, 0]  # x_val_1.shape = (60000, 28, 28)
# x_test_2 = x_test_2[:, 1]

In [27]:
def distances(data1, data2):
    cosine  = []
    euclidean = []
    cityblock = []
    for index, image in enumerate(data1):
        cosine.append(distance.cosine(image, data2[index]))
        euclidean.append(distance.euclidean(image, data2[index]))
        cityblock.append(distance.chebyshev(image, data2[index]))
    return cosine, euclidean, cityblock

x_train_cosine, x_train_euclidean, x_train_cityblock = distances(x_train_1, x_train_2)
x_test_cosine, x_test_euclidean, x_test_cityblock = distances(x_test_1, x_test_2)

In [29]:
df_train = pd.DataFrame(columns=["cosine_distance", "euclidean_distance", "cityblock", "label"])

df_test = pd.DataFrame(columns=["cosine_distance", "euclidean_distance", "cityblock", "label"])

In [30]:
df_train["cosine_distance"] = list(x_train_cosine)
df_train["euclidean_distance"] = list(x_train_euclidean)
df_train["cityblock"] = list(x_train_cityblock)
df_train["label"] = list(y_train)

In [31]:
df_test["cosine_distance"] = list(x_test_cosine)
df_test["euclidean_distance"] = list(x_test_euclidean)
df_test["cityblock"] = list(x_test_cityblock)
df_test["label"] = list(y_test)

In [34]:
df_train.head()

,cosine_distance,euclidean_distance,cityblock,label
0,0.025886,14.759619,0.597386,1
1,0.063717,28.259579,0.743791,1
2,0.087552,24.716150,0.691503,1
3,0.059222,14.122931,0.661438,1
4,0.094167,24.618526,0.627451,1


In [33]:
df_test.head(3)

,cosine_distance,euclidean_distance,cityblock,label
0,0.050155,17.781984,0.836601,1
1,0.098412,26.856802,0.874510,1
2,0.065974,21.101994,0.742484,1


In [36]:
x_train

NameError: name 'x_train' is not defined

In [35]:
df_train["label"]

0       1
1       1
2       1
3       1
4       1
       ..
2195    0
2196    0
2197    0
2198    0
2199    0
Name: label, Length: 2200, dtype: int64

In [42]:
# convert True -> 1 and False -> 0
df_train["label"] = df_train["label"].astype(int)
df_test["label"] = df_test["label"].astype(int)

# features = df_train.drop(columns=["label","difference"]).columns.tolist()

x_train = df_train.drop(columns=["label"])
y_train = df_train["label"]
x_test = df_test.drop(columns=["label"])
y_test = df_test["label"]

In [43]:
xgb_cl = xgb.XGBClassifier(n_estimators=250,
                             max_depth=8,
                             objective= 'binary:logistic',
                             seed=27)

In [44]:
# Fit
xgb_cl.fit(x_train, y_train)

# Predict
y_pred = xgb_cl.predict(x_test)

# Score
accuracy_score(y_test, y_pred)

0.591

In [45]:
from sklearn.metrics import confusion_matrix, classification_report

target_names = ["Mismatched","Matched" ]


print(classification_report(y_pred, y_test, target_names=target_names))

              precision    recall  f1-score   support

  Mismatched       0.60      0.59      0.59       509
     Matched       0.58      0.59      0.59       491

    accuracy                           0.59      1000
   macro avg       0.59      0.59      0.59      1000
weighted avg       0.59      0.59      0.59      1000

